In [1]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
import sys
!{sys.executable} -m pip install gtfparse

  Created wheel for gtfparse: filename=gtfparse-1.2.0-py3-none-any.whl size=11881 sha256=b5c9914eb867c3dee28f96fa30d73dafb706f1029dedcd8a40c8908e852673d8
  Stored in directory: /home/jovyan/.cache/pip/wheels/2d/81/83/85c4108d53b4b2ad3cee5d6f9b7ec612554e132202b8020359
Successfully built gtfparse


In [2]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import urllib.request
from gtfparse import read_gtf

In [3]:
# Setup
data_path = '../data'
gtf_path_gz = data_path + '/Mus_musculus.GRCm38.99.gtf.gz'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'

In [4]:
# Download initial datasets and decompress
if path.exists(gtf_path_gz) is False: 
    urllib.request.urlretrieve('ftp://ftp.ensembl.org/pub/release-99/gtf/mus_musculus/Mus_musculus.GRCm38.99.gtf.gz', gtf_path_gz)
    input = gzip.GzipFile(gtf_path_gz, 'rb')
    s = input.read()
    input.close()
    output = open(gtf_path, 'wb')
    output.write(s)
    output.close()

In [ ]:
df = read_gtf(gtf_path)

In [ ]:
df